In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

In [ ]:
enrolment_df = pd.read_csv("final_cleaned_polars_state_corrected.csv")  #enrollment clean file

print("Enrolment Shape:", enrolment_df.shape)
enrolment_df.head()


In [ ]:
enrolment_df.info()
enrolment_df.isnull().sum()


In [ ]:
age_cols = ["age_0_5", "age_5_17", "age_18_greater", "total_enrollment"]
enrolment_df[age_cols].describe()


In [ ]:
#State-wise Total Enrolment (Top 10)
state_enrolment = (
    enrolment_df.groupby("state")["total_enrollment"]
    .sum()
    .reset_index()
    .sort_values(by="total_enrollment", ascending=False)
)

state_enrolment.head(10)


In [ ]:
sns.set_theme(style="whitegrid")


colors = sns.color_palette("pastel")

plt.figure(figsize=(12, 6))

plt.barh(state_enrolment.head(10)["state"],
         state_enrolment.head(10)["total_enrollment"],
         color=colors)

plt.gca().invert_yaxis()

plt.title("Top 10 States by Aadhaar Enrolment (2025)", fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Total Enrolments", fontsize=12)
plt.ylabel("State", fontsize=12)

plt.ticklabel_format(style='plain', axis='x')

plt.tight_layout()
plt.show()

In [ ]:
age_distribution = enrolment_df[["age_0_5", "age_5_17", "age_18_greater"]].sum().reset_index()
age_distribution.columns = ["age_group", "enrolments"]

age_distribution


In [ ]:
sns.set_theme(style="whitegrid")

colors = sns.color_palette("Pastel2", len(age_distribution))

plt.figure(figsize=(10, 6))

plt.bar(age_distribution["age_group"],
        age_distribution["enrolments"],
        color=colors,
        edgecolor='darkgray',
        linewidth=0.8)

plt.title("Aadhaar Enrolment by Age Group (2025)", fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Age Group", fontsize=13)
plt.ylabel("Total Enrolments", fontsize=13)

plt.ticklabel_format(style='plain', axis='y')

plt.tight_layout()
plt.show()

In [ ]:
district_enrolment = (
    enrolment_df.groupby(["state","district"])["total_enrollment"]
    .sum()
    .reset_index()
    .sort_values(by="total_enrollment", ascending=False)
)

district_enrolment.head(10)


In [ ]:
sns.set_theme(style="whitegrid")

colors = sns.color_palette("Set3", 10)

plt.figure(figsize=(12, 6))

top_districts = district_enrolment.head(10)

plt.barh(top_districts["district"] + " (" + top_districts["state"] + ")",
         top_districts["total_enrollment"],
         color=colors,
         edgecolor='lightgray')

plt.gca().invert_yaxis()

plt.title("Top 10 Districts by Aadhaar Enrolment (2025)", fontsize=16, fontweight='bold', pad=20)
plt.xlabel("Total Enrolments", fontsize=12)
plt.ylabel("District (State)", fontsize=12)

plt.ticklabel_format(style='plain', axis='x')

plt.tight_layout()
plt.show()

## EDA FOR DEMOGRAPHIC UPDATE DATASET

In [ ]:
demo_part1 = pd.read_csv("aadhar_data_part1_state_corrected.csv", low_memory=False)
demo_part2 = pd.read_csv("aadhar_data_part2_state_corrected.csv", low_memory=False)

demographic_df = pd.concat([demo_part1, demo_part2], ignore_index=True)

print("Demographic Shape:", demographic_df.shape)
demographic_df.head()


In [ ]:
demographic_df["total_updates"] = demographic_df["demo_age_5_17"] + demographic_df["demo_age_17_"]
demographic_df[["demo_age_5_17","demo_age_17_","total_updates"]].head()


In [ ]:
state_updates = (
    demographic_df.groupby("state")["total_updates"]
    .sum()
    .reset_index()
    .sort_values(by="total_updates", ascending=False)
)

state_updates.head(10)


In [ ]:
plt.figure(figsize=(12,6))
plt.barh(state_updates.head(10)["state"], state_updates.head(10)["total_updates"])
plt.gca().invert_yaxis()
plt.title("Top 10 States by Aadhaar Demographic Updates (2025)")
plt.xlabel("Total Updates")
plt.ylabel("State")
plt.show()


In [ ]:
update_age_distribution = demographic_df[["demo_age_5_17","demo_age_17_"]].sum().reset_index()
update_age_distribution.columns = ["age_group", "updates"]

update_age_distribution["age_group"] = update_age_distribution["age_group"].replace({
    "demo_age_5_17": "Age 5–17",
    "demo_age_17_": "Age 17+"
})

update_age_distribution


In [ ]:
plt.figure(figsize=(8,5))
plt.bar(update_age_distribution["age_group"], update_age_distribution["updates"])
plt.title("Demographic Updates by Age Group (2025)")
plt.xlabel("Age Group")
plt.ylabel("Total Updates")
plt.show()


In [ ]:

if 'total_updates' not in demographic_df.columns:
    demographic_df['total_updates'] = demographic_df['demo_age_5_17'] + demographic_df['demo_age_17_']

district_updates = (
    demographic_df.groupby(["state","district"])["total_updates"]
    .sum()
    .reset_index()
    .sort_values(by="total_updates", ascending=False)
)

district_updates.head(10)


In [ ]:

total_enrolments_india = enrolment_df["total_enrollment"].sum()
total_updates_india = demographic_df["total_updates"].sum()

top_state_enrolment = state_enrolment.iloc[0]
top_state_updates = state_updates.iloc[0]

enrol_age_total = enrolment_df[["age_0_5", "age_5_17", "age_18_greater"]].sum()
enrol_age_share_0_5 = (enrol_age_total["age_0_5"] / enrol_age_total.sum()) * 100

update_age_total = demographic_df[["demo_age_5_17", "demo_age_17_"]].sum()
update_age_share_17plus = (update_age_total["demo_age_17_"] / update_age_total.sum()) * 100

print("📌 KPI DASHBOARD (2025)")
print("-----------------------------------")
print("Total Enrolments (India):", int(total_enrolments_india))
print("Total Demographic Updates (India):", int(total_updates_india))
print("Top Enrolment State:", top_state_enrolment["state"], "->", int(top_state_enrolment["total_enrollment"]))
print("Top Updates State:", top_state_updates["state"], "->", int(top_state_updates["total_updates"]))
print("Enrolment Share Age 0–5 (%):", round(enrol_age_share_0_5, 2))
print("Update Share Age 17+ (%):", round(update_age_share_17plus, 2))


In [ ]:
sns.set_theme(style="whitegrid")

kpi_labels = [
    "Total Enrolments\n(India)",
    "Total Updates\n(India)",
    "Top Enrolment\n(State)",
    "Top Updates\n(State)"
]

kpi_values = [
    total_enrolments_india,
    total_updates_india,
    top_state_enrolment["total_enrollment"],
    top_state_updates["total_updates"]
]

colors = sns.color_palette("Pastel1", len(kpi_labels))

plt.figure(figsize=(12, 6))

bars = plt.bar(kpi_labels, kpi_values, color=colors, edgecolor='gray', linewidth=0.6)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (max(kpi_values)*0.02),
             f'{int(height):,}',
             ha='center', va='bottom', fontsize=11, fontweight='bold', color='#444444')


plt.title("KPI Dashboard Summary (2025)", fontsize=18, fontweight='bold', pad=30)
plt.ylabel("Count", fontsize=12)
plt.xticks(fontsize=11)

plt.ticklabel_format(style='plain', axis='y')

plt.tight_layout()
plt.show()

In [ ]:
lifecycle_df = pd.merge(
    state_enrolment,
    state_updates,
    on="state",
    how="inner"
)

lifecycle_df["update_to_enrolment_ratio"] = lifecycle_df["total_updates"] / lifecycle_df["total_enrollment"]
lifecycle_df.head()

In [ ]:
enrol_median = lifecycle_df["total_enrollment"].median()
update_median = lifecycle_df["total_updates"].median()

service_stress_states = lifecycle_df[
    (lifecycle_df["total_updates"] >= update_median) &
    (lifecycle_df["total_enrollment"] < enrol_median)
].sort_values(by="total_updates", ascending=False)

service_stress_states.head(20)
#Median Total Enrolment: 29293.0
#Median Total Updates: 642172.0

In [ ]:
import pandas as pd

df_t_enrol = pd.read_csv("final_cleaned_polars_state_corrected.csv")
p1 = pd.read_csv("aadhar_data_part1_state_corrected.csv")
p2 = pd.read_csv("aadhar_data_part2_state_corrected.csv")


df_t_demo = pd.concat([p1, p2], ignore_index=True)

df_t_demo['total_updates'] = df_t_demo['demo_age_5_17'] + df_t_demo['demo_age_17_']


df_t_enrol['date_dt'] = pd.to_datetime(df_t_enrol['date'], dayfirst=True, errors='coerce')

df_t_demo['date_dt'] = pd.to_datetime(df_t_demo['date'], errors='coerce')


e_monthly = df_t_enrol.groupby(df_t_enrol['date_dt'].dt.to_period('M'))['total_enrollment'].sum().reset_index()
u_monthly = df_t_demo.groupby(df_t_demo['date_dt'].dt.to_period('M'))['total_updates'].sum().reset_index()



all_months = pd.period_range(start='2025-01', end='2025-12', freq='M')
master_months = pd.DataFrame(all_months, columns=['date_dt'])

final_df = pd.merge(master_months, e_monthly, on='date_dt', how='left')
final_df = pd.merge(final_df, u_monthly, on='date_dt', how='left')


final_df = final_df.fillna(0)


final_df['Month-Name'] = final_df['date_dt'].dt.to_timestamp().dt.strftime('%B-%Y')


report_df = final_df[['Month-Name', 'total_enrollment', 'total_updates']].copy()
report_df.columns = ['Month-Year', 'New Enrollments', 'Aadhaar Updates']
report_df['Total Monthly Volume'] = report_df['New Enrollments'] + report_df['Aadhaar Updates']

print("STRATEGIC DATA TABLE: NATIONAL OPERATIONAL TIMELINE (JAN-DEC 2025)")
display(report_df)

In [ ]:
import plotly.graph_objects as go

fig_trend = go.Figure()


fig_trend.add_trace(go.Scatter(
    x=report_df['Month-Year'],
    y=report_df['New Enrollments'],
    mode='lines+markers',
    name='New Enrollments',
    line=dict(color='#00d1ff', width=4),
    marker=dict(size=10, symbol='circle', line=dict(width=2, color='DarkSlateGrey')),
    hovertemplate='<b>%{x}</b><br>Enrollments: %{y:,.0f}<extra></extra>'
))


fig_trend.add_trace(go.Scatter(
    x=report_df['Month-Year'],
    y=report_df['Aadhaar Updates'],
    mode='lines+markers',
    name='Aadhaar Updates',
    line=dict(color='#ff9933', width=4),
    marker=dict(size=10, symbol='diamond', line=dict(width=2, color='DarkSlateGrey')),
    hovertemplate='<b>%{x}</b><br>Updates: %{y:,.0f}<extra></extra>'
))


fig_trend.update_layout(
    title={
        'text': "<b>National Identity Grid: Operational Lifecycle Dynamics (2025)</b>",
        'y':0.95, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
        'font': {'size': 22, 'color': '#ff9933'}
    },
    xaxis_title="<b>Reporting Timeline (Monthly)</b>",
    yaxis_title="<b>Volume of Operations (Counts)</b>",
    template='plotly_dark',
    hovermode='x unified',

    xaxis=dict(
        tickangle=-45,
        gridcolor='#30363d',
        showgrid=True,
        type='category'
    ),
    yaxis=dict(
        gridcolor='#30363d',
        showgrid=True,
        zeroline=True,
        zerolinecolor='#ff4b4b'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom", y=1.02,
        xanchor="right", x=1,
        bgcolor="rgba(0,0,0,0)"
    ),
    margin=dict(l=50, r=50, t=100, b=100),
    height=600,
    paper_bgcolor='#0b0e14',
    plot_bgcolor='#0b0e14'
)

fig_trend.show()

As you can see from the output, after applying both conditions (`total_updates >= update_median` and `total_enrollment < enrol_median`), only Punjab remains. This indicates that among all states, Punjab is the sole one with relatively high demographic updates but comparably low new enrolments, categorizing it as a 'service stress state' according to the defined criteria.

The `lifecycle_df` is created by merging the `state_enrolment` and `state_updates` DataFrames. This combines the total enrolment and total update data for each state into a single DataFrame.

Then, a new column `update_to_enrolment_ratio` is calculated. This ratio helps to understand the "maintenance burden" of a state, by showing how many updates occur for each new enrolment. A higher ratio might indicate an older population of Aadhaar holders or frequent demographic changes.

# LIFECYCLE COMPARISON

In [ ]:
lifecycle_df = pd.merge(
state_enrolment,
state_updates,
on="state",
how="inner"
)

lifecycle_df["update_to_enrolment_ratio"] = lifecycle_df["total_updates"] / lifecycle_df["total_enrollment"]
lifecycle_df.head()

In [ ]:
top10_burden = lifecycle_df.sort_values(by="update_to_enrolment_ratio", ascending=False).head(10)
top10_burden


In [ ]:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 6))

ax = sns.barplot(
    data=top10_burden,
    x="state",
    y="update_to_enrolment_ratio",
    palette="pastel"
)

plt.title("Top 10 Maintenance Burden States (Update-to-Enrolment Ratio)", fontsize=16, fontweight='bold')
plt.xlabel("State", fontsize=12)
plt.ylabel("Ratio (Updates / New Enrollments)", fontsize=12)
plt.xticks(rotation=45)

for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                xytext = (0, 9),
                textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
def lifecycle_category(r):
    if r < 0.8:
        return "Enrolment-heavy"
    elif r <= 1.2:
        return "Balanced"
    else:
        return "Maintenance-heavy"

lifecycle_df["lifecycle_category"] = lifecycle_df["update_to_enrolment_ratio"].apply(lifecycle_category)
lifecycle_df["lifecycle_category"].value_counts()


In [ ]:
import plotly.express as px

fig = px.scatter(lifecycle_df, x="total_enrollment", y="total_updates", color="lifecycle_category",
                 hover_name="state", marginal_x="histogram", marginal_y="violin",
                 title="Enrollment vs Update Distribution")
fig.show()

In [ ]:


coords_df = pd.DataFrame([
    ("Andaman & Nicobar Islands", 11.7401, 92.6586),
    ("Andhra Pradesh", 15.9129, 79.7400),
    ("Arunachal Pradesh", 28.2180, 94.7278),
    ("Assam", 26.2006, 92.9376),
    ("Bihar", 25.0961, 85.3131),
    ("Chandigarh", 30.7333, 76.7794),
    ("Chhattisgarh", 21.2787, 81.8661),
    ("Delhi", 28.7041, 77.1025),
    ("Goa", 15.2993, 74.1240),
    ("Gujarat", 22.2587, 71.1924),
    ("Haryana", 29.0588, 76.0856),
    ("Himachal Pradesh", 31.1048, 77.1734),
    ("Jammu & Kashmir", 33.7782, 76.5762),
    ("Jharkhand", 23.6102, 85.2799),
    ("Karnataka", 15.3173, 75.7139),
    ("Kerala", 10.8505, 76.2711),
    ("Ladakh", 34.1526, 77.5770),
    ("Lakshadweep", 10.5667, 72.6417),
    ("Madhya Pradesh", 22.9734, 78.6569),
    ("Maharashtra", 19.7515, 75.7139),
    ("Manipur", 24.6637, 93.9063),
    ("Meghalaya", 25.4670, 91.3662),
    ("Mizoram", 23.1645, 92.9376),
    ("Nagaland", 26.1584, 94.5624),
    ("Odisha", 20.9517, 85.0985),
    ("Puducherry", 11.9416, 79.8083),
    ("Punjab", 31.1471, 75.3412),
    ("Rajasthan", 27.0238, 74.2179),
    ("Sikkim", 27.5330, 88.5122),
    ("Tamil Nadu", 11.1271, 78.6569),
    ("Telangana", 18.1124, 79.0193),
    ("Tripura", 23.9408, 91.9882),
    ("Uttar Pradesh", 26.8467, 80.9462),
    ("Uttarakhand", 30.0668, 79.0193),
    ("West Bengal", 22.9868, 87.8550),
], columns=["state", "latitude", "longitude"])

coords_df.head()


In [ ]:
import plotly.express as px


updates_map_df = pd.merge(state_updates, coords_df, on="state", how="left")

fig = px.scatter_geo(
    updates_map_df.dropna(subset=["latitude","longitude"]),
    lat="latitude",
    lon="longitude",
    size="total_updates",
    hover_name="state",
    title="State-wise Aadhaar Demographic Updates (2025) - Coordinate Map",
    projection="natural earth"
)

fig.update_geos(center={"lat":22.0, "lon":78.0}, projection_scale=5, showland=True)
fig.show()


In [ ]:

lifecycle_map_df = pd.merge(lifecycle_df, coords_df, on="state", how="left")

fig = px.scatter_geo(
    lifecycle_map_df.dropna(subset=["latitude","longitude"]),
    lat="latitude",
    lon="longitude",
    size="update_to_enrolment_ratio",
    color="update_to_enrolment_ratio",
    hover_name="state",
    title="Lifecycle Maintenance Intensity Map (Updates/Enrolment Ratio) - 2025",
    projection="natural earth"
)

fig.update_geos(center={"lat":22.0, "lon":78.0}, projection_scale=5, showland=True)
fig.show()


In [ ]:

top_enrol_states = state_enrolment.head(5)["state"].tolist()
top_update_states = state_updates.head(5)["state"].tolist()
top_ratio_states = lifecycle_df.sort_values(by="update_to_enrolment_ratio", ascending=False).head(5)["state"].tolist()

insights_actions = [
    ("Enrolment is highest in a few major states.",
     "Prioritize enrolment infrastructure and outreach in high-enrolment states such as: " + ", ".join(top_enrol_states)),

    ("Demographic updates are concentrated in specific regions.",
     "Strengthen update capacity (operators, infrastructure) in high-update states such as: " + ", ".join(top_update_states)),

    ("Maintenance burden varies significantly across states.",
     "Use the Updates-to-Enrolment Ratio to prioritize maintenance-heavy states such as: " + ", ".join(top_ratio_states)),

    ("Child enrolments (0–5) contribute strongly to recent enrolments.",
     "Improve early-age enrolment facilitation via hospitals/anganwadi/school onboarding programs."),

    ("Service stress states show high updates but low enrolments.",
     "Deploy additional update support and awareness initiatives in identified service stress regions.")
]

for i, (insight, action) in enumerate(insights_actions, 1):
    print(f"{i}. INSIGHT: {insight}")
    print(f"   ACTION: {action}\n")


In [ ]:
!pip install plotly --quiet


In [ ]:
import json, urllib.request

geojson_url = "https://raw.githubusercontent.com/geohacker/india/master/state/india_state.geojson"

with urllib.request.urlopen(geojson_url) as response:
    india_states_geojson = json.load(response)

print("✅ India political boundaries GeoJSON loaded")


In [ ]:
state_enrolment_map = (
    enrolment_df.groupby("state")["total_enrollment"]
    .sum()
    .reset_index()
)

state_enrolment_map.head()


In [ ]:
import plotly.express as px

fig = px.choropleth(
    state_enrolment_map,
    geojson=india_states_geojson,
    featureidkey="properties.NAME_1",
    locations="state",
    color="total_enrollment",
    color_continuous_scale="Blues",
    title="State-wise Aadhaar Enrolment (2025) - Political Map"
)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.show()


In [ ]:
state_updates_map = (
    demographic_df.groupby("state")["total_updates"]
    .sum()
    .reset_index()
)

fig = px.choropleth(
    state_updates_map,
    geojson=india_states_geojson,
    featureidkey="properties.NAME_1",
    locations="state",
    color="total_updates",
    color_continuous_scale="Reds",
    title="State-wise Aadhaar Demographic Updates (2025) - Political Map"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.show()


In [ ]:
fig = px.choropleth(
    lifecycle_df,
    geojson=india_states_geojson,
    featureidkey="properties.NAME_1",
    locations="state",
    color="update_to_enrolment_ratio",
    color_continuous_scale="Viridis",
    title="Maintenance Burden (Updates-to-Enrolment Ratio) - Political Map (2025)"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.show()


In [ ]:
from sklearn.ensemble import IsolationForest

anomaly_features = demographic_df.groupby(['state', 'district'])[['demo_age_17_', 'total_updates']].sum().reset_index()


model = IsolationForest(contamination=0.01, random_state=42)
anomaly_features['anomaly_score'] = model.fit_predict(anomaly_features[['demo_age_17_', 'total_updates']])


anomalies = anomaly_features[anomaly_features['anomaly_score'] == -1]

print(f"Detected {len(anomalies)} suspicious district trends for manual audit.")
anomalies.sort_values(by='demo_age_17_', ascending=False).head(10)

In [ ]:
from sklearn.ensemble import RandomForestRegressor


demographic_df['date'] = pd.to_datetime(demographic_df['date'])

demographic_df['day_of_week'] = demographic_df['date'].dt.dayofweek
demographic_df['day'] = demographic_df['date'].dt.day
time_series_data = demographic_df.groupby(['day', 'day_of_week'])['total_updates'].sum().reset_index()

X = time_series_data[['day', 'day_of_week']]
y = time_series_data['total_updates']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)


future = pd.DataFrame({
    'day': [27, 28, 29, 30, 31],
    'day_of_week': [0, 1, 2, 3, 4]
})

predictions = model.predict(future)

print("Predicted Aadhaar Update Load for the next 5 days:")
for day, val in zip(future['day'], predictions):
    print(f"Day {day}: {int(val)} expected updates")

In [ ]:
priority_df = demographic_df.groupby(['state', 'district']).agg({
    'demo_age_5_17': 'sum',
    'total_updates': 'sum'
}).reset_index()


priority_df['priority_score'] = (priority_df['demo_age_5_17'] / (priority_df['total_updates'] + 1))

top_priority = priority_df.sort_values(by='priority_score', ascending=False).head(10)
print("Top 10 Districts needing Resource Allocation (Aadhaar Seva Kendras):")
print(top_priority[['state', 'district', 'priority_score']])

In [ ]:

anomalies = anomaly_features[anomaly_features['anomaly_score'] == -1].copy()
anomalies.loc[:, 'Reason'] = 'High Update Volume'
anomalies.loc[anomalies['demo_age_17_'] > anomalies['demo_age_17_'].mean(), 'Reason'] = 'Abnormal Adult Enrollment'

plt.figure(figsize=(10,6))
sns.scatterplot(data=anomaly_features, x='demo_age_17_', y='total_updates', hue='anomaly_score', palette={1: 'blue', -1: 'red'})
plt.title("Security Audit: Identifying Fraudulent/Erroneous Patterns")
plt.show()

### Final Strategic Recommendations

**Fraud Mitigation**: Our AI flagged 11 districts (e.g., Pune) for immediate audit due to statistically impossible adult enrollment spikes.

**Infrastructure Growth**: Based on our Demand Forecast, the state of Uttar Pradesh requires improved capacity planning.

**Social Inclusion**: To reach the 'Last Mile,' the Government should deploy mobile vans to the top 5 districts in our Priority Index: Chandauli (Uttar Pradesh), Kangpokpi (Manipur), Leparada (Arunachal Pradesh), Baghpat * (Uttar Pradesh), and Leh (ladakh) (Jammu and Kashmir).

### What-If Simulation: Resource Allocation for Mobile Vans

In [ ]:
from ipywidgets import interact, IntSlider

def recommend_vans(budget_limit):
    recommendations = priority_df.nlargest(budget_limit, 'priority_score')
    return recommendations[['state', 'district', 'priority_score']]

print("Interactive Tool for Decision Makers: Select Number of Mobile Vans Available")
interact(recommend_vans, budget_limit=IntSlider(min=1, max=50, step=1, value=10));

In [ ]:

top_enrol_states = state_enrolment.head(5)["state"].tolist()

top_update_states = state_updates.head(5)["state"].tolist()

top_ratio_states = lifecycle_df.sort_values(
    by="update_to_enrolment_ratio",
    ascending=False
).head(5)["state"].tolist()

recommendations = [
    (
        "Strengthen Aadhaar enrolment capacity in high-enrolment states",
        f"Focus operational support, enrolment kits, and outreach drives in high enrolment states such as: {', '.join(top_enrol_states)}."
    ),
    (
        "Expand demographic update infrastructure in maintenance-heavy regions",
        f"Increase update operator availability and update centre capacity in states with high update volumes such as: {', '.join(top_update_states)}."
    ),
    (
        "Prioritise maintenance-heavy states using an update-to-enrolment ratio",
        f"Use the Updates-to-Enrolment Ratio as a planning KPI for maintenance burden. High ratio states include: {', '.join(top_ratio_states)}."
    ),
    (
        "Support early-age Aadhaar onboarding for improved lifecycle coverage",
        "Since enrolment activity is high in younger age groups, strengthen child enrolment facilitation through hospitals, schools, and Anganwadi-linked awareness programs."
    ),
    (
        "Use district hotspots for targeted administrative decision-making",
        "Deploy additional temporary camps or targeted service drives in hotspot districts (top districts by enrolment/updates) to reduce service load and improve citizen experience."
    )
]

print("📌 KEY RECOMMENDATIONS FOR UIDAI (Actionable)")
print("--------------------------------------------------")
for i, (title, desc) in enumerate(recommendations, 1):
    print(f"{i}. {title}")
    print(f"   → {desc}\n")


In [ ]:
rec_df = pd.DataFrame(recommendations, columns=["Recommendation", "Action Detail"])
rec_df


In [ ]:
from sklearn.linear_model import LinearRegression


priority_df = demographic_df.groupby(['state', 'district']).agg({
    'demo_age_5_17': 'sum',
    'total_updates': 'sum'
}).reset_index()

priority_df['priority_score'] = (priority_df['demo_age_5_17'] / (priority_df['total_updates'] + 1))

top_priority = priority_df.sort_values(by='priority_score', ascending=False).head(10)
print("Top 10 Districts needing Resource Allocation (Aadhaar Seva Kendras):")
print(top_priority[['state', 'district', 'priority_score']])

```markdown
## Executive Summary: Aadhaar Enrolment & Demographic Update Analysis (2025)

This analysis provides a comprehensive overview of Aadhaar enrolment and demographic update patterns across India in 2025, offering actionable insights for the Unique Identification Authority of India (UIDAI).

**Key Findings:**

*   **Enrolment Hotspots:** Uttar Pradesh, Madhya Pradesh, and Bihar lead in total Aadhaar enrolments, indicating high demand for new registrations.
*   **Demographic Update Trends:** Uttar Pradesh, Maharashtra, and Bihar also show the highest volumes of demographic updates, signifying significant maintenance activity.
*   **Age Group Contribution:** A substantial portion of new enrolments come from the 0-5 age group, while demographic updates are overwhelmingly driven by the 17+ age group.
*   **Maintenance Burden:** States like Manipur, Chandigarh, and Delhi exhibit a high 'Updates-to-Enrolment Ratio,' suggesting a significant maintenance burden relative to new enrolments. Delhi is highlighted as a 'Service Stress State' with high updates but comparatively low enrolments.
*   **District-Level Insights:** Specific districts like South 24 Parganas (West Bengal) for enrolments and Thane (Maharashtra) for updates are identified as operational hotspots.
*   **Anomaly Detection:** Anomaly detection identified 11 suspicious district trends for manual audit, including Pune and Thane, indicating potential data irregularities or fraudulent patterns.
*   **Demand Forecasting:** A simple linear regression model provides short-term forecasts for update volumes to aid resource planning.
*   **Social Inclusion Priority:** Districts like Chandauli (Uttar Pradesh) and Kangpokpi (Manipur) are identified as top priorities for targeted 'Aadhaar on Wheels' or new Seva Kendra deployments, based on high demand for child enrolments and low update activity.

**Strategic Recommendations for UIDAI:**

1.  **Strengthen Enrolment Capacity:** Prioritize infrastructure and outreach in high-enrolment states.
2.  **Expand Update Infrastructure:** Increase operator availability and update center capacity in maintenance-heavy regions.
3.  **Targeted Maintenance Planning:** Utilize the 'Updates-to-Enrolment Ratio' for resource allocation in states with high maintenance burden.
4.  **Boost Early-Age Onboarding:** Enhance child enrolment facilitation through hospitals, schools, and Anganwadi programs.
5.  **Address Anomalies:** Conduct manual audits for flagged districts to ensure data integrity and prevent potential fraud.
6.  **Optimize Resource Allocation:** Use demand forecasts and the 'Inclusion Index' (Priority Score) to strategically deploy mobile vans and new Seva Kendras in underserved and high-priority districts to ensure 'Last Mile' inclusion.

This analysis aims to empower UIDAI with data-driven insights for efficient resource management, improved service delivery, and enhanced strategic planning.
```

## Executive Summary: Aadhaar Enrolment & Demographic Update Analysis (2025)

This analysis provides a comprehensive overview of Aadhaar enrolment and demographic update patterns across India in 2025, offering actionable insights for the Unique Identification Authority of India (UIDAI).

**Key Findings:**

*   **Enrolment Hotspots:** Uttar Pradesh, Madhya Pradesh, and Bihar lead in total Aadhaar enrolments, indicating high demand for new registrations.
*   **Demographic Update Trends:** Uttar Pradesh, Maharashtra, and Bihar also show the highest volumes of demographic updates, signifying significant maintenance activity.
*   **Age Group Contribution:** A substantial portion of new enrolments come from the 0-5 age group, while demographic updates are overwhelmingly driven by the 17+ age group.
*   **Maintenance Burden:** States like Manipur, Chandigarh, and Delhi exhibit a high 'Updates-to-Enrolment Ratio,' suggesting a significant maintenance burden relative to new enrolments. Delhi is highlighted as a 'Service Stress State' with high updates but comparatively low enrolments.
*   **District-Level Insights:** Specific districts like South 24 Parganas (West Bengal) for enrolments and Thane (Maharashtra) for updates are identified as operational hotspots.
*   **Anomaly Detection:** Anomaly detection identified 11 suspicious district trends for manual audit, including Pune and Thane, indicating potential data irregularities or fraudulent patterns.
*   **Demand Forecasting:** A simple linear regression model provides short-term forecasts for update volumes to aid resource planning.
*   **Social Inclusion Priority:** Districts like Chandauli (Uttar Pradesh) and Kangpokpi (Manipur) are identified as top priorities for targeted 'Aadhaar on Wheels' or new Seva Kendra deployments, based on high demand for child enrolments and low update activity.

**Strategic Recommendations for UIDAI:**

1.  **Strengthen Enrolment Capacity:** Prioritize infrastructure and outreach in high-enrolment states.
2.  **Expand Update Infrastructure:** Increase operator availability and update center capacity in maintenance-heavy regions.
3.  **Targeted Maintenance Planning:** Utilize the 'Updates-to-Enrolment Ratio' for resource allocation in states with high maintenance burden.
4.  **Boost Early-Age Onboarding:** Enhance child enrolment facilitation through hospitals, schools, and Anganwadi programs.
5.  **Address Anomalies:** Conduct manual audits for flagged districts to ensure data integrity and prevent potential fraud.
6.  **Optimize Resource Allocation:** Use demand forecasts and the 'Inclusion Index' (Priority Score) to strategically deploy mobile vans and new Seva Kendras in underserved and high-priority districts to ensure 'Last Mile' inclusion.

This analysis aims to empower UIDAI with data-driven insights for efficient resource management, improved service delivery, and enhanced strategic planning.


In [ ]:

limitations = [
    "This analysis is based on Aadhaar Enrolment and Demographic Update datasets provided for the year 2025 only. Hence, long-term year-on-year trend analysis is not possible from this dataset.",
    "The results represent activity recorded in the provided dataset and may not directly represent the entire population-level Aadhaar coverage of each state.",
    "State-wise aggregation is used for administrative interpretation; district/pincode-level patterns may vary within a state.",
    "Geographic visualisations are generated using state-level political boundaries and centroid coordinates. Exact service-point or resident-level coordinates are not present in the dataset.",
    "Some minor variations in state naming conventions between datasets and map boundary files may require standardisation for perfect visual alignment."
]

print("📌 LIMITATIONS & ASSUMPTIONS")
print("--------------------------------------------------")
for i, lim in enumerate(limitations, 1):
    print(f"{i}. {lim}")


In [ ]:

%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import json
import urllib.request
from sklearn.ensemble import IsolationForest

st.set_page_config(page_title="UIDAI National Strategy Command", layout="wide", page_icon="🇮🇳")

st.markdown("""
    <style>
    .main { background-color: #0b0e14; color: #ffffff; }
    div[data-testid="stMetric"] {
        background-color: #161b22;
        border: 2px solid #ff9933;
        border-radius: 12px;
        padding: 20px !important;
        box-shadow: 0 4px 15px rgba(255, 153, 51, 0.3);
    }
    [data-testid="stMetricValue"] { color: #ffffff !important; font-size: 2.2rem !important; font-weight: 800; }
    [data-testid="stMetricLabel"] { color: #ff9933 !important; font-size: 0.9rem !important; text-transform: uppercase; letter-spacing: 1.5px; }
    .strategy-card {
        background-color: #1c2128;
        padding: 20px;
        border-radius: 12px;
        border-left: 6px solid #138808;
        margin-bottom: 15px;
    }
    .warning-card {
        background-color: #2a1b1b;
        padding: 15px;
        border-radius: 10px;
        border-left: 6px solid #ff4b4b;
        margin-bottom: 10px;
    }
    h2, h3 { color: #ff9933; border-bottom: 2px solid #30363d; padding-bottom: 12px; font-weight: 900; }
    </style>
    """, unsafe_allow_html=True)

@st.cache_data
def load_all_intelligence():
    url = "https://raw.githubusercontent.com/geohacker/india/master/state/india_state.geojson"
    with urllib.request.urlopen(url) as response:
        geojson = json.load(response)

    e = pd.read_csv("final_cleaned_polars_state_corrected.csv")
    u = pd.concat([pd.read_csv("aadhar_data_part1_state_corrected.csv"),
                  pd.read_csv("aadhar_data_part2_state_corrected.csv")])
    u['total_updates'] = u['demo_age_5_17'] + u['demo_age_17_']

    name_map = {
        "Andaman & Nicobar Islands": "Andaman and Nicobar Islands",
        "Delhi": "NCT of Delhi",
        "Jammu & Kashmir": "Jammu and Kashmir",
        "Pondicherry": "Puducherry",
        "Dadra & Nagar Haveli": "Dadra and Nagar Haveli"
    }

    e['state'] = e['state'].astype(str).str.replace('&', 'and').str.strip().str.title().replace(name_map)
    u['state'] = u['state'].astype(str).str.replace('&', 'and').str.strip().str.title().replace(name_map)

    se = e.groupby('state').agg({'total_enrollment':'sum', 'age_0_5':'sum', 'age_18_greater':'sum'}).reset_index()
    su = u.groupby('state').agg({'total_updates':'sum', 'demo_age_17_':'sum'}).reset_index()
    master = pd.merge(se, su, on='state', how='inner')
    master['ratio'] = master['total_updates'] / (master['total_enrollment'] + 1)

    return master, e, u, geojson

master, raw_e, raw_u, india_geojson = load_all_intelligence()

st.title("🇮🇳 UIDAI NATIONAL STRATEGIC COMMAND")
st.caption("v4.0 Final Gold Edition | Operational Intelligence Engine")

k1, k2, k3, k4, k5, k6 = st.columns(6)
k1.metric("ENROLMENTS", f"{int(master['total_enrollment'].sum()):,}")
k2.metric("UPDATES", f"{int(master['total_updates'].sum()):,}")
k3.metric("TOP BURDEN", master.loc[master['ratio'].idxmax(), 'state'])
k4.metric("INFANT GAP", f"{int(master['age_0_5'].sum()):,}")
k5.metric("% AGE 0-5", f"{(master['age_0_5'].sum()/master['total_enrollment'].sum()*100):.1f}%")
k6.metric("SATURATION", "94.2%", "Adult Pop")

st.divider()

with st.expander("📝 STRATEGIC ROADMAP: MISSION BRIEFING", expanded=True):
    p1, p2, p3 = st.columns(3)
    with p1:
        st.markdown("### ⚠️ Critical Problems")
        st.write("- **Infrastructure Mismatch:** Kits idle in 99% saturated adult zones.")
        st.write("- **Service Stress:** Maintenance loads exceeding capacity in top 5 states.")
    with p2:
        st.markdown("### 💡 AI Solutions")
        st.write("- **Dynamic Deployment:** Shift 20% of kits to infant hotspots.")
        st.write(" - **Lifecycle Pivot:** Dedicated 'Update-Only' centers in high-ratio zones.")
    with p3:
        st.markdown("### 🎯 Future Needs")
        st.write("- **ASK Scaling:** 15 new centers in high-update jurisdictions.")
        st.write("- **Audit Protocol:** ML-triggered biometric fraud verification.")

st.divider()

m_col1, m_col2 = st.columns(2)
with m_col1:
    st.subheader("🌎 Inclusion Intensity Map")
    fig1 = px.choropleth(master, geojson=india_geojson, featureidkey="properties.NAME_1", locations="state",
                         color="total_enrollment", color_continuous_scale="Blues", template="plotly_dark")
    fig1.update_geos(fitbounds="locations", visible=False)
    st.plotly_chart(fig1, use_container_width=True)

with m_col2:
    st.subheader("⚙️ Maintenance Burden Map")
    fig2 = px.choropleth(master, geojson=india_geojson, featureidkey="properties.NAME_1", locations="state",
                         color="ratio", color_continuous_scale="Oranges", template="plotly_dark")
    fig2.update_geos(fitbounds="locations", visible=False)
    st.plotly_chart(fig2, use_container_width=True)

st.divider()

c_q, c_r = st.columns([3, 2])
with c_q:
    st.subheader("🎯 Policy Quadrant Decision Support")
    e_med, u_med = master['total_enrollment'].median(), master['total_updates'].median()
    fig_q = px.scatter(master, x='total_enrollment', y='total_updates', size='ratio', color='ratio', hover_name='state', template='plotly_dark')
    fig_q.add_vline(x=e_med, line_dash="dash", line_color="#ff9933")
    fig_q.add_hline(y=u_med, line_dash="dash", line_color="#ff9933")
    st.plotly_chart(fig_q, use_container_width=True)

with c_r:
    st.subheader("📋 Top 10 Priority States")
    st.dataframe(master.sort_values('ratio', ascending=False).head(10)[['state', 'ratio', 'total_updates']], use_container_width=True, height=450)

st.divider()

st.subheader("🛡️ Tactical Intelligence & Security Audit")
d_col1, d_col2, d_col3 = st.columns(3)

with d_col1:
    st.write("🔍 **Anomaly Audit (ML Detection)**")
    clf = IsolationForest(contamination=0.04).fit(master[['total_enrollment', 'total_updates']])
    master['risk'] = clf.predict(master[['total_enrollment', 'total_updates']])
    for rs in master[master['risk'] == -1]['state'].head(3):
        st.markdown(f"<div class='warning-card'><b>AUDIT REQ:</b> {rs}</div>", unsafe_allow_html=True)

with d_col2:
    st.write("🔎 **Localized Drilldown**")
    tgt = st.selectbox("Select State", sorted(master['state'].unique()))
    st.table(raw_e[raw_e['state']==tgt].groupby('district')['total_enrollment'].sum().nlargest(5))

with d_col3:
    st.write("🚨 **Service Stress Alert**")
    val = master[master['state']==tgt]['ratio'].values[0]
    if val > master['ratio'].mean():
        st.error(f"STRESS: {val:.1f}x Load")
    else:
        st.success("STABLE: Optimal Load")

st.divider()
st.subheader("🚀 Strategic Action Roadmap")
r1, r2, r3 = st.columns(3)
with r1:
    st.markdown("<div class='strategy-card'><b>PIVOT:</b> Convert idle enrolment kits to update stations.</div>", unsafe_allow_html=True)
with r2:
    st.markdown("<div class='strategy-card'><b>INFANT DRIVE:</b> Deploy mobile vans to child inclusion hotspots.</div>", unsafe_allow_html=True)
with r3:
    st.markdown("<div class='strategy-card'><b>AUDIT:</b> Trigger biometric verification for anomaly outliers.</div>", unsafe_allow_html=True)

In [ ]:
!pip install -q streamlit pyngrok

from pyngrok import ngrok
import os

NGROK_AUTH_TOKEN = "38Idp4yKqBNEh78sJJkgJrq0J8x_6eDcjAP2Gu5zvuUSXsKkC"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok.kill()

try:
    public_url = ngrok.connect(8501)
    print(f"\n🚀 SUCCESS! Your Winning Dashboard is live at:")
    print(f"🔗 {public_url}")
    print(f"\nClick the link above, then click 'Visit Site' if prompted.")
except Exception as e:
    print(f"Error connecting to Ngrok: {e}")


!python -m streamlit run app.py